In [1]:
import pandas as pd
from math import sqrt
from sklearn import tree
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('apartment_sales_spb.csv')
df

,Тип,Цена (₽),Комнаты,Площадь (м²),Жилая площадь (м²),Площадь кухни (м²),Потолок,Этаж,Всего этажей в доме,Стены,Район,Метро,Расстояние до метро (км)
0,Квартира,8200000,1,40.85,18.7,10.9,NaN,4,10,Панельный,Калининский,NaN,NaN
1,Квартира,13599000,4,94.80,80.8,8.0,2.80,2,5,Кирпичный,Выборгский,Лесная,0.6
2,Квартира,7700000,4,50.00,44.0,6.0,3.00,5,5,Индивидуальный,Московский,Московская,NaN
3,Квартира,9390000,2,58.50,28.5,17.6,2.56,18,24,Индивидуальный,Невский,NaN,NaN
4,Студия,3500000,0,24.10,15.0,2.0,2.50,1,16,Индивидуальный,Всеволожский,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8784,Квартира,9500000,2,56.40,32.5,11.7,NaN,2,18,Монолит,Всеволожский,Улица Дыбенко,1.4
8785,Квартира,124000000,3,248.00,179.9,10.0,NaN,10,15,Панельный,Петроградский,Чкаловская,NaN
8786,Студия,4700000,0,24.10,18.0,2.0,2.65,2,12,Индивидуальный,Красносельский,NaN,NaN
8787,Студия,4580000,0,26.70,14.0,NaN,NaN,13,16,Кирпично-Монолитный,Всеволожский,Девяткино,1.6


In [3]:
df.describe()

,Цена (₽),Комнаты,Площадь (м²),Жилая площадь (м²),Площадь кухни (м²),Потолок,Этаж,Всего этажей в доме,Расстояние до метро (км)
count,8.789000e+03,8789.000000,8789.000000,8789.000000,8219.000000,7184.000000,8789.000000,8789.000000,6196.000000
mean,1.479865e+07,1.815337,58.534262,32.732928,11.693149,3.001048,6.533280,12.234725,6.264187
std,2.024017e+08,1.637538,56.272066,41.557760,9.123161,9.330513,5.319572,7.148840,21.231738
min,3.700000e+05,0.000000,8.000000,1.000000,0.100000,1.000000,1.000000,1.000000,0.100000
25%,5.900000e+06,1.000000,34.700000,16.900000,7.100000,2.600000,3.000000,5.000000,0.900000
50%,8.290000e+06,2.000000,47.200000,27.000000,10.000000,2.700000,5.000000,10.000000,1.800000
75%,1.321800e+07,3.000000,67.000000,39.300000,14.000000,2.800000,9.000000,17.000000,4.600000
max,1.890000e+10,40.000000,1980.000000,1571.000000,471.000000,450.000000,34.000000,36.000000,1319.400000


### Обработка данных

Удалим строки с отсутствующими значениями в признаках ```Метро```, ```Район``` и ```Стены```. Удалим ошибочные данные (значения в признаке ```Потолок``` больше 300 метров) и выбросы в ```Цена (₽)``` и ```Комнаты```.

In [4]:
df.dropna(inplace=True,subset=['Метро'])
df.dropna(inplace=True,subset=['Район'])
df.dropna(inplace=True,subset=['Стены'])
df['Потолок'] = df['Потолок'].fillna(0)
df = df[df['Расстояние до метро (км)'] != 1319.4]
df = df[df['Комнаты'] != 40]
df = df[df['Потолок'] < 300]
df = df[df['Цена (₽)'] != 18900000000]
df.reset_index(inplace=True, drop=True)

Отсутствующие значения в ```Площадь кухни (м²)``` заменим на 0, а в ```Потолок``` и ```Расстояние до метро (км)``` на средние значения этих признаков.

In [5]:
df['Площадь кухни (м²)'] = df['Площадь кухни (м²)'].fillna(0)
df.loc[df['Потолок'] == 0, 'Потолок'] = df['Потолок'][df['Потолок'] != 0].mean()

In [6]:
df.fillna(df.mean(numeric_only=True), inplace=True)
# df.fillna(method = 'pad', inplace=True)
df = df.convert_dtypes()
df

,Тип,Цена (₽),Комнаты,Площадь (м²),Жилая площадь (м²),Площадь кухни (м²),Потолок,Этаж,Всего этажей в доме,Стены,Район,Метро,Расстояние до метро (км)
0,Квартира,13599000,4,94.8,80.8,8.0,2.8,2,5,Кирпичный,Выборгский,Лесная,0.6
1,Квартира,7700000,4,50.0,44.0,6.0,3.0,5,5,Индивидуальный,Московский,Московская,6.048374
2,Квартира,8300000,2,50.6,29.5,8.6,2.57,7,13,Кирпичный,Красносельский,Проспект Ветеранов,4.4
3,Студия,5600000,0,23.9,17.8,0.0,3.02,24,24,Монолит,Калининский,Лесная,2.8
4,Квартира,4000000,1,41.9,18.7,10.7,2.55,7,10,600.11 серия,Фрунзенский,Купчино,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6681,Квартира,13980000,3,57.0,42.1,4.0,2.75,7,7,Сталинский,Московский,Парк Победы,0.5
6682,Квартира,6900000,1,34.1,16.21,9.1,2.785352,13,23,Кирпично-Монолитный,Приморский,Комендантский проспект,3.7
6683,Квартира,9500000,2,56.4,32.5,11.7,2.785352,2,18,Монолит,Всеволожский,Улица Дыбенко,1.4
6684,Квартира,124000000,3,248.0,179.9,10.0,2.785352,10,15,Панельный,Петроградский,Чкаловская,6.048374


Категориальные признаки ```Тип```, ```Район```, ```Метро``` и ```Стены``` закодируем значениями от 0 до n-1, где n - количество различных меток.

In [7]:
labelencoder_area = LabelEncoder()
df['Район'] = labelencoder_area.fit_transform(df['Район'])
labelencoder_metro = LabelEncoder()
df['Метро'] = labelencoder_metro.fit_transform(df['Метро'])
labelencoder_type = LabelEncoder()
df['Тип'] = labelencoder_type.fit_transform(df['Тип'])
labelencoder_walls = LabelEncoder()
df['Стены'] = labelencoder_walls.fit_transform(df['Стены'])

In [8]:
le_name_mapping = dict(zip(labelencoder_area.classes_, labelencoder_area.transform(labelencoder_area.classes_)))
print(le_name_mapping)

{'Адмиралтейский': 0, 'Василеостровский': 1, 'Волосовский': 2, 'Всеволожский': 3, 'Выборгский': 4, 'Гатчинский': 5, 'Калининский': 6, 'Кингисеппский': 7, 'Кировский': 8, 'Колпинский': 9, 'Красногвардейский': 10, 'Красносельский': 11, 'Кронштадтский': 12, 'Курортный': 13, 'Ломоносовский': 14, 'Лужский': 15, 'Московский': 16, 'Невский': 17, 'Петроградский': 18, 'Петродворцовый': 19, 'Приморский': 20, 'Приозерский': 21, 'Пушкинский': 22, 'Тосненский': 23, 'Фрунзенский': 24, 'Центральный': 25}


In [9]:
df

,Тип,Цена (₽),Комнаты,Площадь (м²),Жилая площадь (м²),Площадь кухни (м²),Потолок,Этаж,Всего этажей в доме,Стены,Район,Метро,Расстояние до метро (км)
0,0,13599000,4,94.8,80.8,8.0,2.8,2,5,15,4,25,0.6
1,0,7700000,4,50.0,44.0,6.0,3.0,5,5,13,16,30,6.048374
2,0,8300000,2,50.6,29.5,8.6,2.57,7,13,15,11,51,4.4
3,1,5600000,0,23.9,17.8,0.0,3.02,24,24,17,6,25,2.8
4,0,4000000,1,41.9,18.7,10.7,2.55,7,10,5,24,22,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6681,0,13980000,3,57.0,42.1,4.0,2.75,7,7,22,16,39,0.5
6682,0,6900000,1,34.1,16.21,9.1,2.785352,13,23,14,20,20,3.7
6683,0,9500000,2,56.4,32.5,11.7,2.785352,2,18,17,3,63,1.4
6684,0,124000000,3,248.0,179.9,10.0,2.785352,10,15,19,18,67,6.048374


### Обучение моделей

In [10]:
y = df['Цена (₽)']
X = df.drop(columns=['Цена (₽)'])

In [31]:
kfold = KFold(n_splits=4, random_state=1, shuffle=True)

lr = LinearRegression()
dtr = tree.DecisionTreeRegressor()
rfr = RandomForestRegressor()

print('Linear Regression:')
print(' R^2 =', sum(cross_val_score(lr, X, y, cv=kfold, scoring='r2'))/4)
print(' MAE =', sum(abs(cross_val_score(lr, X, y, cv=kfold, scoring='neg_mean_absolute_error')))/4)
print(' RMSE =', sum(abs(cross_val_score(lr, X, y, cv=kfold, scoring='neg_root_mean_squared_error')))/4)
print()
print('Decision Tree Regressor:')
print(' R^2 =', sum(cross_val_score(dtr, X, y, cv=kfold, scoring='r2'))/4)
print(' MAE =', sum(abs(cross_val_score(dtr, X, y, cv=kfold, scoring='neg_mean_absolute_error')))/4)
print(' RMSE =', sum(abs(cross_val_score(dtr, X, y, cv=kfold, scoring='neg_root_mean_squared_error')))/4)
print()
print('Random Forest Regressor:')
print(' R^2 =', sum(cross_val_score(rfr, X, y, cv=kfold, scoring='r2'))/4)
print(' MAE =', sum(abs(cross_val_score(rfr, X, y, cv=kfold, scoring='neg_mean_absolute_error')))/4)
print(' RMSE =', sum(abs(cross_val_score(rfr, X, y, cv=kfold, scoring='neg_root_mean_squared_error')))/4)

Linear Regression:
 R^2 = 0.5169598732172254
 MAE = 4525423.746647669
 RMSE = 12905081.935404357

Decision Tree Regressor:
 R^2 = 0.2688053014029337
 MAE = 4217104.893828081
 RMSE = 14849892.784741301

Random Forest Regressor:
 R^2 = 0.6125350812956204
 MAE = 3129546.514009265
 RMSE = 11576223.920556664


Наибольшая точность у алгоритма случайного леса. Подберем гиперпараметры с помощью GridSearch, чтобы улучшить результат этой модели.

In [21]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size= 0.2, random_state=7)

parameters = {
    'max_depth': [5, 10, 20, 50, 100],
    'n_estimators': [10, 20, 40, 50, 100]
}
model = RandomForestRegressor()

grid_search = GridSearchCV(model, parameters, cv=4)
grid_search.fit(xtrain, ytrain)

GridSearchCV(cv=4, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [5, 10, 20, 50, 100],
                         'n_estimators': [10, 20, 40, 50, 100]})

In [28]:
best_model = grid_search.best_estimator_
best_model.fit(xtrain, ytrain)
predictions = best_model.predict(xtest)

Самый высокий результат получен при количестве деревьев равном 50 и максимальной глубине деревьев 20.

In [29]:
best_model

RandomForestRegressor(max_depth=20, n_estimators=50)

In [144]:
print(' R^2 =', r2_score(ytest, predictions))
print(' MAE =', mean_absolute_error(ytest, predictions))
print(' RMSE =', sqrt(mean_squared_error(ytest, predictions)))

 R^2 = 0.6798933455041556
 MAE = 3242536.883330964
 RMSE = 10137707.6641787
